In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import copy
options = Options()
options.headless = True

In [2]:
browser = webdriver.Chrome(executable_path="./chromedriver.exe", options=options)

In [4]:
browser.get('https://library.daejin.ac.kr/data_data_list.mir?search_keyword_type1=text&search_keyword1=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C&srch_condi_01=AND&search_keyword_type2=author&search_keyword2=&srch_condi_02=AND&search_keyword_type3=subject&search_keyword3=&srch_condi_03=%E2%96%B2&select_current_sort_type=&select_current_sort_order=&select_current_per_page_num=10&basic_list=list&current_per_page_num=10&current_sort_type=&current_sort_order=&MAXI=1000&mloc_code=DJUL&facet_limit_search_field_code=&scope_code=&mtype_code=M&type_code=')

In [8]:
#//*[@id="content_data"]/form/div[2]/table/tbody/tr[1]/td[3]/a
result1 = browser.find_element_by_xpath('//*[@id="content_data"]/form/div[2]/table/tbody/tr[1]/td[3]/a')

In [10]:
result1.text

'도서 데이터 분석 전문가 가이드 =The guide for advanced data analytics professiona'

In [11]:
result1.get_attribute('href')

'https://library.daejin.ac.kr/data_data_list.mir?search_keyword_type1=text&search_keyword1=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C&srch_condi_01=AND&search_keyword_type2=author&search_keyword2=&srch_condi_02=AND&search_keyword_type3=subject&search_keyword3=&srch_condi_03=%E2%96%B2&select_current_sort_type=&select_current_sort_order=&select_current_per_page_num=10&basic_list=list&current_per_page_num=10&current_sort_type=&current_sort_order=&MAXI=1000&mloc_code=DJUL&facet_limit_search_field_code=&scope_code=&mtype_code=M&type_code=#link'

In [12]:
result1.click()

In [13]:
browser.current_url

'https://library.daejin.ac.kr/data_view.mir?current_sort_type=&current_sort_order=&basic_list=list&current_per_page_num=10&search_keyword1=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C&search_keyword2=&search_keyword3=&search_keyword_type1=text&search_keyword_type2=author&search_keyword_type3=subject&srch_condi_01=AND&srch_condi_02=AND&page_num=1&scope_code=&mtype_code=M&type_code=&search_re_yn=&search_keyword_re=&search_keyword_type_re=&search_keyword_type1_re_add=&search_keyword1_re_add=&facet_limit_search_field_code=&facet_limit_search_start_pub_year=&facet_limit_search_end_pub_year=&rno=403080&hloc_code=DJUL&all_hloc_code='